In [ ]:
import io
import os
import shutil

import filetype

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import fitz

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from skimage import io, transform

# test

In [ ]:
path = "/Users/romanmalinowski/Documents/Manga"
manga = "DANDADAN"

# Don't know why but the pdf has blank pages, but converting to RGB fixes it
def convert(i):
    if i.mode == "P":
        return i.convert('L')
    return i

for tome in ["T07"]:
    print(tome)
    n_tome = tome.split("T")[-1]

    im_files = os.listdir(os.path.join(path, manga, tome))
    im_files = [i for i in im_files if (".jpg" in i.lower()) or (".jpeg" in i.lower()) or (".png" in i.lower())]
    im_files.sort()

    images = [convert(Image.open(os.path.join(path, manga, tome, i))) for i in im_files]

In [ ]:
for k, image in enumerate(images):
    if image._size[1] < image._size[0]:
        print(k)

In [ ]:
images[37]

In [ ]:
images[38]._size

In [ ]:
im = np.asarray(images[38]).astype(np.uint8)
im.shape

In [ ]:
middle_col = im.shape[1]//2
margin = 30

fig, ax = plt.subplots(1,1,figsize=(20,20))

ax.imshow(im[:,middle_col-margin:middle_col+margin], cmap="gray", vmin=0, vmax=255)

In [ ]:
fake_i = 30
im_l = np.asarray(images[fake_i]).astype(np.uint8)
im_r = np.asarray(images[fake_i+1]).astype(np.uint8)

im_fake = np.hstack((im_l, im_r))

In [ ]:
middle_col = im_fake.shape[1]//2
margin = 30

fig, ax = plt.subplots(1,1,figsize=(20,20))

ax.imshow(im_fake[:,middle_col-margin:middle_col+margin], cmap="gray", vmin=0, vmax=255)

In [ ]:
np.array(images[38])[300-10:300+10, 1200-10:1200+10]

# Bases for dataset
[Torch tuto](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html): `https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html`

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
class MangaPagesDataset(torch.utils.data.Dataset):
    """Manga Pages dataset."""

    def __init__(self, csv_file, root_dir, seed=42, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)
    
    # TODO
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.dataframe.iloc[idx, 0])
        image = io.imread(img_name)
        is_double_page = bool(self.dataframe.iloc[idx, 1])
        sample = {'image': image, 'is_double_page': is_double_page}

        if self.transform:
            sample = self.transform(sample)

        return sample

## Extracting PDF

In [ ]:
root_path = "/Users/romanmalinowski/Documents/Manga"

In [ ]:
manga = "DANDADAN"

list_tomes = [t for t in os.listdir(os.path.join(root_path, manga)) if os.path.isdir(os.path.join(root_path, manga, t)) if t[0]=="T"]
list_tomes

In [ ]:
list_tomes = ['T01', 'T02', 'T03', 'T04', 'T05', 'T13', 'T14', 'T15']

In [ ]:
for tome in list_tomes:
    for page in os.listdir(os.path.join(root_path, manga, tome)):
        src = os.path.join(root_path, manga, tome, page)
        if page == ".DS_Store":
            continue
        kind = filetype.guess(src)
        if kind.mime.split("/")[0]!="image":
            continue
        
        dst = os.path.join("/Users/romanmalinowski/Code/Classifier_manga/data/images", f"{manga}_{tome}_{page}")
        shutil.copyfile(src=src, dst=dst)

In [ ]:
manga = "Berserk"

list_tomes = [t.split(".pdf")[0] for t in os.listdir(os.path.join("/Users/romanmalinowski/Documents/Manga/", manga)) if ".pdf" in t]
list_tomes

In [ ]:
for tome in list_tomes:
    
    path = f"/Users/romanmalinowski/Documents/Manga/{manga}/{tome}.pdf"
    pdf_file = fitz.open(path)
    print(tome)
    
    for page_number, page in enumerate(pdf_file): 
        image_list = page.get_images()
        for image_index, img in enumerate(page.get_images(), start=1):
            xref = img[0] 
            # extract image bytes 
            base_image = pdf_file.extract_image(xref)
            image_bytes = base_image["image"]
 
            # Create a PIL Image object from the image bytes
            pil_image = Image.open(io.BytesIO(image_bytes))
            dst = os.path.join("/Users/romanmalinowski/Code/Classifier_manga/data/images", f"{manga}_{tome}_{page_number}.png")
            pil_image.save(dst)

In [ ]:
for tome in list_tomes:
    
    path = f"/Users/romanmalinowski/Documents/Manga/{manga}/{tome}.pdf"
    pdf_file = fitz.open(path)
    print(tome)
    
    for page_number, page in enumerate(pdf_file): 
        image_list = page.get_images()
        for image_index, img in enumerate(page.get_images(), start=1):
            xref = img[0] 
            # extract image bytes 
            base_image = pdf_file.extract_image(xref)
            image_bytes = base_image["image"]

            # Create a PIL Image object from the image bytes
            pil_image = Image.open(io.BytesIO(image_bytes))
            dst = os.path.join("/Users/romanmalinowski/Code/Classifier_manga/data/images", f"{manga}_{tome}_{page_number}.png")
            pil_image.save(dst)

# Renaming and verifying that there is some double pages

In [ ]:
print(f"{len([k for k in os.listdir('data/images') if k[0]!='.'])} images in total")

In [ ]:
np.unique([k.split("_")[0] for k in os.listdir("data/images") if k[0]!="."])

In [ ]:
name = "Abara-Deluxe-2023"
list_files = [k for k in os.listdir('data/images') if k[0]!='.']
list_name = [k for k in list_files if name in k]

In [ ]:
list_name[:5]

In [ ]:
for k in list_name:
    new_name = k.split("_")[0]+"_"+k.split("_")[1].replace("-", "_") 
    #if os.path.exists(os.path.join("data/images", new_name)):
    #   print("Error, file already exists")
    print(k, "-", new_name)
    #os.rename(os.path.join("data/images", k), os.path.join("data/images", new_name))

In [ ]:
name = "Abara-Deluxe-2023"
list_files = [k for k in os.listdir('data/images') if k[0]!='.']
list_name = [k for k in list_files if name in k]

## Counting double pages ratio

In [ ]:
list_tomes = np.unique([k.split("_")[1] for k in list_name])

double_pages = {t: [] for t in list_tomes}
for tome in list_tomes:
    pages = [k for k in list_name if tome in k]
    for page in pages:
        double_pages[tome] += [Image.open(os.path.join("data/images", page)).size]

for tome in list_tomes:
    count_double_pages = 0
    for size in double_pages[tome]:
        if size[0] > size[1]:
            count_double_pages += 1
    print(f"{tome}: {count_double_pages / len(double_pages[tome]) *100:.2f} %")

# Creating a dataframe counting info on the data

In [ ]:
list_files = [k for k in os.listdir('data/images') if k[0]!='.']

df = pd.DataFrame(columns=["File_name", "Manga", "Volume"], index=np.arange(len(list_files)).astype(int))

In [ ]:
for i, file in enumerate(list_files):
    split = file.split("_")
    df.loc[i, ["File_name", "Manga", "Volume"]] = [file, split[0], split[1]]

In [ ]:
df = df.sort_values(axis=0, by=["File_name"], ignore_index=True)
df = df.drop(columns=["index"])

In [ ]:
df.to_csv("data/images_data.csv", index=False)

In [ ]:
df_bis = pd.read_csv("data/images_data.csv")
df_bis

In [ ]:
object???